In [23]:
import xarray as xr
import numpy as np
import pandas as pd
from functools import reduce

lat = 2160
lon = 4320
ocean_label = "ocean"

def gfrac_to_excel(out_file_name, input_data, year, columns_name, input_label, index_data, column_data, agg):
    dataframes = []
    for i, ngfbfc in enumerate(input_label):
        frames = []
        n = 0
        df_ngfbfc = input_data[ngfbfc]
        while n < year:
            df = df_ngfbfc.isel(time=n).to_dataframe()
            df_table = pd.pivot_table(df, index=index_data, columns=column_data, aggfunc=agg, fill_value=0)
            df_table = df_table.stack(level=0, future_stack=True)

            df_table.columns = pd.to_datetime(df_table.columns, format='%d/%m/%Y %H.%M.%S').year
            df_index = df_table.reset_index()
            df_index.rename(columns={'level_1': columns_name}, inplace=True)
            frames.append(df_index)
            n += 1
            
        df_result = reduce(lambda left, right: pd.merge(left, right, on=[index_data[0], columns_name]), frames)
        df_result = df_result.replace([np.inf, -np.inf], np.nan)
        df_result = df_result.fillna(0)
        dataframes.append(df_result)

    merged_df = pd.concat(dataframes, axis=0, ignore_index=True).fillna(0)
    merged_df.to_excel(f"/{out_file_name}.xlsx", index=False)

    return merged_df

In [2]:
GFRAC = xr.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC.nc", engine="netcdf4")
GFRAC = GFRAC.drop_sel(NGFBFC=b'grass                                             ')
GFRAC = GFRAC.drop_sel(NGFBFC=b'Grains (biofuel)                                  ')
GFRAC = GFRAC.drop_sel(NGFBFC=b'Oil crops (biofuel)                               ')
GFRAC = GFRAC.drop_sel(NGFBFC=b'Sugar cane (biofuel)                              ')
GFRAC = GFRAC.drop_sel(NGFBFC=b'Woody biofuel                                     ')
GFRAC = GFRAC.drop_sel(NGFBFC=b'Non-woody biofuel                                 ')
gfrac_ngfbfc32i = [element.strip() for element in GFRAC.coords['NGFBFC'].data.astype('str').tolist()]

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx", engine="openpyxl")
luh_static = xr.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/CCODE_RASTER.nc")

ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')
ccode_convert = np.zeros((lat, lon), dtype="<U64")
cname_dict = {}

for idx, ccode in enumerate(ccode_iso):
    cname_dict[ccode] = cname_iso[idx]

for x in range(lat):
    for y in range(lon):
        if (ccode_worldwide_int[x][y] in cname_dict.keys()):
            ccode_convert[x][y] = cname_dict[ccode_worldwide_int[x][y]]
        else:
            ccode_convert[x][y] = ocean_label

country_coords = xr.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": GFRAC .coords["longitude"].to_numpy(),
                                  "latitude": GFRAC .coords["latitude"].to_numpy()})

GFRAC = GFRAC.assign_coords(NGFBFC=gfrac_ngfbfc32i)

gfracarea = np.zeros((32, 3, 2160, 4320), dtype="float32")
for i, ngfbfc in enumerate(gfrac_ngfbfc32i):
    for n in range(3):
        gfracarea[i][n] = GFRAC['GFRAC'].isel(NGFBFC=i, time=n)

gfrac_newnetcdf2 = xr.Dataset(
    coords={
        "time": pd.date_range(start='1970-01-01', end='1980-01-01', freq='5YS'),
        "latitude": GFRAC.coords["latitude"].to_numpy(),
        "longitude": GFRAC.coords["longitude"].to_numpy(),
        "country_name": country_coords['country']
    })
coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, gfracarea[i]) for i, ngfbc in enumerate(gfrac_ngfbfc32i)
}
gfrac_newnetcdf2 = gfrac_newnetcdf2.assign(data_vars)
gfrac_newnetcdf2

<xarray.Dataset>
Dimensions:                              (longitude: 4320, latitude: 2160,
                                          time: 3)
Coordinates:
  * longitude                            (longitude) float64 -180.0 ... 180.0
  * latitude                             (latitude) float64 89.96 ... -89.96
  * time                                 (time) datetime64[ns] 1970-01-01 ......
    country_name                         (latitude, longitude) <U64 'ocean' ....
Data variables: (12/32)
    RF Wheat                             (time, latitude, longitude) float32 ...
    RF Rice                              (time, latitude, longitude) float32 ...
    RF Maize                             (time, latitude, longitude) float32 ...
    RF Tropical cereals                  (time, latitude, longitude) float32 ...
    RF Other temperate cereals           (time, latitude, longitude) float32 ...
    RF Pulses                            (time, latitude, longitude) float32 ...
    ...                                   ...
    IR Tropical roots & tubers           (time, latitude, longitude) float32 ...
    IR Sugar crops                       (time, latitude, longitude) float32 ...
    IR Oil & palm fruit                  (time, latitude, longitude) float32 ...
    IR Vegetables & fruits               (time, latitude, longitude) float32 ...
    IR Other non-food & luxury & spices  (time, latitude, longitude) float32 ...
    IR Plant based fibres                (time, latitude, longitude) float32 ...

In [3]:
gfrac_newnetcdf2.to_netcdf("/GFRACarea_32.NC", mode='w', format="NETCDF4")

In [21]:
dataframes = []
for i, ngfbfc in enumerate(gfrac_ngfbfc32i):
    frames = []
    n = 0
    while n < 3:
        df_ngfbfc = gfrac_newnetcdf2[ngfbfc]
        df = df_ngfbfc.isel(time=n).to_dataframe()
        df_table = pd.pivot_table(df, index='country_name', columns='time', aggfunc='sum', fill_value=0)
        df_table = df_table.stack(level=0, future_stack=True)

        df_table.columns = pd.to_datetime(df_table.columns, format='%d/%m/%Y %H.%M.%S').year
        df_index = df_table.reset_index()
        df_index.rename(columns={'level_1': 'NGFBFC'}, inplace=True)
        frames.append(df_index)
        n += 1
        
    df_result = reduce(lambda left, right: pd.merge(left, right, on=['country_name', 'NGFBFC']), frames)
    df_result = df_result.replace([np.inf, -np.inf], np.nan)
    df_result = df_result.fillna(0)
    dataframes.append(df_result)

merged_df = pd.concat(dataframes, axis=0, ignore_index=True).fillna(0)
merged_df.to_excel("/GFRACarea_32_concat.xlsx", index=False)

In [ ]:
filtred_Ccode = country_code[["ISO Country", "IMAGE Region Name"]]
region_merge = pd.merge(left=merged_df, right=filtred_Ccode, left_on="country_name", right_on="ISO Country")
region_merge

0          Rest S. Asia
1             C. Europe
2             N. Africa
3               Oceania
4        Rest S. Africa
             ...       
6459    Rest C. America
6460          N. Africa
6461        Middle East
6462     Rest S. Africa
6463     Rest S. Africa
Name: IMAGE Region Name, Length: 6464, dtype: object

In [24]:
gfrac_to_excel(out_file_name="GFRACarea_32_concat", year=3, input_data=gfrac_newnetcdf2, input_label=gfrac_ngfbfc32i, columns_name="NGFBFC", index_data=['country_name'], column_data=['time'], agg='sum')

time,country_name,NGFBFC,1970,1975,1980
0,Afghanistan,RF Wheat,376.393341,644.176147,614.554688
1,Albania,RF Wheat,29.774336,25.185055,21.619812
2,Algeria,RF Wheat,454.539490,462.072144,415.679382
3,American Samoa,RF Wheat,0.000000,0.000000,0.000000
4,Angola,RF Wheat,3.268323,1.803113,1.371239
...,...,...,...,...,...
6491,Western Sahara,IR Plant based fibres,0.000000,0.000000,0.000000
6492,Yemen,IR Plant based fibres,0.453346,0.559732,0.567619
6493,Zambia,IR Plant based fibres,0.169107,0.222499,0.290833
6494,Zimbabwe,IR Plant based fibres,0.117285,0.152618,0.197425
